In [4]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.applications as applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

2025-02-26 10:41:21.469165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740546681.494047   13495 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740546681.501720   13495 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 10:41:21.528145: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 5
TRAIN_DIR = "/home/shihas/dl_projects/Crop Disease Detection/split_dataset/train"
VAL_DIR = "/home/shihas/dl_projects/Crop Disease Detection/split_dataset/val"
TEST_DIR = "/home/shihas/dl_projects/Crop Disease Detection/split_dataset/test"
NUM_CLASSES = 39

In [6]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

I0000 00:00:1740546687.019938   13495 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2603 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [7]:
def preprocess_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, tf.cast(label, tf.int64)

def load_dataset(directory):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory, image_size=IMG_SIZE, batch_size=None
    ).map(preprocess_image)
    
    dataset = dataset.filter(lambda img, label: tf.math.reduce_all(tf.math.is_finite(img)))
    return dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset = load_dataset(TRAIN_DIR).map(lambda x, y: (data_augmentation(x), y))
val_dataset = load_dataset(VAL_DIR)
test_dataset = load_dataset(TEST_DIR)


Found 43030 files belonging to 39 classes.
Found 9213 files belonging to 39 classes.
Found 9243 files belonging to 39 classes.


In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Extract labels using a small subset
small_train_labels = np.concatenate([
    labels.numpy() for _, labels in train_dataset.take(500)  # Use only 500 batches
])

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(small_train_labels),
    y=small_train_labels
)

# Convert to TensorFlow dictionary format
class_weight_dict = {int(cls): weight for cls, weight in zip(np.unique(small_train_labels), class_weights)}

print("✅ Fast Approximate Class Weights:", class_weight_dict)


In [ ]:
def create_model():
    base_model = applications.ResNet34(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = True  # fine-tuning

    for layer in base_model.layers[:-20]:  #  Freeze most layers, fine-tune last 20
        layer.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = create_model()
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 39)             │        10,023 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,122,279 (92.02 MB)

 Trainable params: 9,465,895 (36.11 MB)

 Non-trainable params: 14,656,384 (55.91 MB)

In [17]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[lr_scheduler, checkpoint, early_stopping],
    class_weight = class_weight_dict
)

Epoch 1/5
   2689/Unknown 515s 187ms/step - accuracy: 0.7171 - loss: 4.3680

2025-02-26 09:50:27.832709: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[6,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[6,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-02-26 09:50:28.217041: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[6,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[6,128,28,28]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivati

   2690/Unknown 530s 192ms/step - accuracy: 0.7171 - loss: 4.3675

2025-02-26 09:50:37.190250: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
/home/shihas/miniconda3/envs/tf-env/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
2025-02-26 09:51:32.075265: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_c

2690/2690 ━━━━━━━━━━━━━━━━━━━━ 592s 215ms/step - accuracy: 0.7172 - loss: 4.3670 - val_accuracy: 0.9510 - val_loss: 1.1369 - learning_rate: 1.0000e-04
Epoch 2/5
2690/2690 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.9373 - loss: 0.9389

...

2690/2690 ━━━━━━━━━━━━━━━━━━━━ 588s 219ms/step - accuracy: 0.9529 - loss: 0.3879 - val_accuracy: 0.9721 - val_loss: 0.2289 - learning_rate: 1.0000e-04
Epoch 4/5
2690/2690 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.9648 - loss: 0.2505

2025-02-26 10:19:27.726937: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2025-02-26 10:19:27.727189: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9186869457731331283
2025-02-26 10:19:27.727197: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17830693944403754443
2025-02-26 10:19:27.727219: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11189440087847831648
2025-02-26 10:20:09.223279: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9186869457731331283
2025-02-26 10:20:09.223318: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17830693944403754443


2690/2690 ━━━━━━━━━━━━━━━━━━━━ 528s 196ms/step - accuracy: 0.9648 - loss: 0.2505 - val_accuracy: 0.9758 - val_loss: 0.1705 - learning_rate: 1.0000e-04
Epoch 5/5
2690/2690 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.9705 - loss: 0.1994

2025-02-26 10:28:13.835506: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9186869457731331283
2025-02-26 10:28:13.835546: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17830693944403754443
2025-02-26 10:28:13.835566: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11189440087847831648
2025-02-26 10:28:55.764377: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9186869457731331283


2690/2690 ━━━━━━━━━━━━━━━━━━━━ 528s 196ms/step - accuracy: 0.9705 - loss: 0.1994 - val_accuracy: 0.9781 - val_loss: 0.1566 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 5.


In [18]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"✅ Test Accuracy: {test_acc:.4f}")

model.save("crop_disease_model.keras")
print("✅ Model saved successfully!")

    577/Unknown 47s 81ms/step - accuracy: 0.9736 - loss: 0.1715

2025-02-26 10:29:46.798617: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[11,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[11,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-02-26 10:29:47.187289: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[11,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[11,128,28,28]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActi

578/578 ━━━━━━━━━━━━━━━━━━━━ 51s 88ms/step - accuracy: 0.9736 - loss: 0.1715
✅ Test Accuracy: 0.9754
✅ Model saved successfully!
